# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
import datetime

def imread(path):
    from PIL import Image
    return np.array(Image.open(path))

def imresize(img, size):
    from PIL import Image
    return np.array(Image.fromarray(img).resize(size)) #documentation: frommarray - Creates an image memory from an object exporting the array interface

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines()) #readlines() stores each line in a list
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 40 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
# x = 30 # No. of frames images
# y = 120 # Width of the image
# z = 120 # height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    #img_idx = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    while True:
        t = np.random.permutation(folder_list) #permutes a sequence randomly with the given input
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    resized_image = (image*255).astype(np.uint8) #normalizing and converting the images to type uint8 from float32 as imresize() does not accept float32
                    resized_image = imresize(resized_image, (y,z)) 
                    
                    batch_data[folder,idx,:,:,0] = (resized_image[:,:,0]) 
                    batch_data[folder,idx,:,:,1] = (resized_image[:,:,1]) 
                    batch_data[folder,idx,:,:,2] = (resized_image[:,:,2])
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(t) - (batch_size*num_batches)
            for batch in range(num_batches):
                  batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                  batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
                  for folder in range(batch_size): # iterate over the batch_size
                      imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                      for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                          image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                          
                          #crop the images and resize them. Note that the images are of 2 different shape 
                          #and the conv3D will throw error if the inputs in a batch have different shapes
                          
                          resized_image = (image*255).astype(np.uint8)
                          resized_image = imresize(resized_image, (y,z)) 
                          
                          batch_data[folder,idx,:,:,0] = (resized_image[:,:,0])
                          batch_data[folder,idx,:,:,1] = (resized_image[:,:,1])
                          batch_data[folder,idx,:,:,2] = (resized_image[:,:,2])
                        
                      batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                  yield batch_data, batch_labels



Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


# 3D Models:

# Model 0

Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.


Inputs: 
x = 15
y = 100
z = 100

batch_size = 30
epochs = 15

In [8]:
x = 15 #passing only 15 inputs
y = 100 #image height
z = 100 #image width

num_epochs = 15
batch_size = 30

In [9]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU,Dropout, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

#write your model here
Input_shape = (15, 100, 100, 3) #15 training images of size (100, 100) with 3 channels

model = Sequential()
#first layer
model.add(Conv3D(16, (3,3,3), padding='same', input_shape = Input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D((2,2,2), padding = "same"))

#2nd layer
model.add(Conv3D(32, (3, 3,3), padding = "same"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D((2,2,2), padding = "same"))
#model.add(Dropout(0.5))

#3rd layer
model.add(Conv3D(64, (3, 3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D((2,2,2), padding = "same"))

#4th layer
model.add(Conv3D(128, (3, 3,3), padding = "same"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D((2,2,2), padding = "same"))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(128))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(5))
model.add(Activation('softmax'))

In [10]:
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.001) #using adam optimizer with a very low learning rate
model.compile(optimizer = optimiser, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 15, 100, 100, 16)  1312      
                                                                 
 activation (Activation)     (None, 15, 100, 100, 16)  0         
                                                                 
 batch_normalization (BatchN  (None, 15, 100, 100, 16)  64       
 ormalization)                                                   
                                                                 
 max_pooling3d (MaxPooling3D  (None, 8, 50, 50, 16)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 8, 50, 50, 32)     13856     
                                                                 
 activation_1 (Activation)   (None, 8, 50, 50, 32)     0

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init_0' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/' #creating the model name with the current time stamp
    
if not os.path.exists(model_name):
    os.mkdir(model_name) #creating the path if it does not already exist
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = False, save_weights_only = False, mode = 'auto', save_freq = 'epoch') #monitor is used to check specific metrics (val_loss in this case); 
#verbose displays messages; save_best_only will save only the best possible model out all
#save_weights_only: if true, only the weights will be saved.
#mode: as per documentation: In auto mode, the mode is set to max if the quantities monitored are 'acc' or start with 'fmeasure' and are set to min for the rest of the quantities.
#save_freq: when using 'epoch', the model is saved after each epoch

LR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2, cooldown = 1, verbose = 1)
callbacks_list = [checkpoint, LR]

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [14]:
model.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs = num_epochs, verbose = 1,  #steps_per_epoch: The number of batches to run during each
                    callbacks = callbacks_list, validation_data = val_generator, #callback performs various activities at different stages. We have assigned checkpoint and LR. Basically it saves the model and prints the metrics
                    validation_steps = validation_steps, class_weight = None, workers = 1, initial_epoch = 0)

Source path =  Project_data/train ; batch size = 30
Epoch 1/15
23/23 [==============================] - ETA: 0s - loss: 1.6057 - categorical_accuracy: 0.4103 Source path =  Project_data/val ; batch size = 30

Epoch 1: saving model to model_init_0_2022-07-1206_46_11.972365/model-00001-1.60572-0.41026-56.77660-0.22000.h5
23/23 [==============================] - 320s 14s/step - loss: 1.6057 - categorical_accuracy: 0.4103 - val_loss: 56.7766 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 2/15
23/23 [==============================] - ETA: 0s - loss: 1.8508 - categorical_accuracy: 0.3913
Epoch 2: saving model to model_init_0_2022-07-1206_46_11.972365/model-00002-1.85084-0.39130-14.66368-0.30000.h5
23/23 [==============================] - 37s 2s/step - loss: 1.8508 - categorical_accuracy: 0.3913 - val_loss: 14.6637 - val_categorical_accuracy: 0.3000 - lr: 0.0010
Epoch 3/15
23/23 [==============================] - ETA: 0s - loss: 2.1931 - categorical_accuracy: 0.2029
Epoch 3: saving mod

# Model 1

Inputs: 

Changing the number of inputs

x = 30
y = 100
z = 100

batch_size = 30
epochs = 15

In [15]:
x = 30
y = 100
z = 100

num_epochs = 15
batch_size = 30

In [16]:
Input_shape = (30, 100, 100, 3)

model_1 = Sequential()
#first layer
model_1.add(Conv3D(16, (3,3,3), padding='same', input_shape = Input_shape))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D((2, 2,2), padding = "same"))

#2nd layer
model_1.add(Conv3D(32, (3, 3,3), padding = "same"))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D((2, 2,2), padding = "same"))

#3rd layer
model_1.add(Conv3D(64, (3, 3,3), padding='same'))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D((2, 2,2), padding = "same"))

#4th layer
model_1.add(Conv3D(128, (3, 3,3), padding = "same"))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D((2, 2,2), padding = "same"))

model_1.add(Flatten())
model_1.add(Dense(256))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.25))

model_1.add(Dense(128))
model_1.add(Activation("relu"))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.25))

model_1.add(Dense(5))
model_1.add(Activation('softmax'))

In [17]:
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.001) #write your optimizer
model_1.compile(optimizer=optimiser, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print(model_1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 30, 100, 100, 16)  1312      
                                                                 
 activation_7 (Activation)   (None, 30, 100, 100, 16)  0         
                                                                 
 batch_normalization_6 (Batc  (None, 30, 100, 100, 16)  64       
 hNormalization)                                                 
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 15, 50, 50, 16)   0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 15, 50, 50, 32)    13856     
                                                                 
 activation_8 (Activation)   (None, 15, 50, 50, 32)   

In [18]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [19]:
model_name = 'model_init_1' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = False, save_weights_only = False, mode = 'auto', save_freq = 'epoch')

LR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2, cooldown = 1, verbose = 1)
callbacks_list = [checkpoint, LR]

In [20]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [21]:
model_1.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs = num_epochs, verbose = 1, 
                    callbacks = callbacks_list, validation_data = val_generator, 
                    validation_steps = validation_steps, class_weight = None, workers = 1, initial_epoch = 0)

Source path =  Project_data/train ; batch size = 30
Epoch 1/15
23/23 [==============================] - ETA: 0s - loss: 1.4840 - categorical_accuracy: 0.4646 Source path =  Project_data/val ; batch size = 30

Epoch 1: saving model to model_init_1_2022-07-1206_46_11.972365/model-00001-1.48401-0.46456-56.26824-0.23000.h5
23/23 [==============================] - 638s 28s/step - loss: 1.4840 - categorical_accuracy: 0.4646 - val_loss: 56.2682 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/15
23/23 [==============================] - ETA: 0s - loss: 1.7441 - categorical_accuracy: 0.4203
Epoch 2: saving model to model_init_1_2022-07-1206_46_11.972365/model-00002-1.74407-0.42029-27.81855-0.10000.h5
23/23 [==============================] - 75s 3s/step - loss: 1.7441 - categorical_accuracy: 0.4203 - val_loss: 27.8185 - val_categorical_accuracy: 0.1000 - lr: 0.0010
Epoch 3/15
23/23 [==============================] - ETA: 0s - loss: 1.8536 - categorical_accuracy: 0.3478
Epoch 3: saving mod

# Model 2

Inputs:
x = 30
y = 100
z = 100

Changing batch size and number of epochs


batch_size = 45
epochs = 20

In [22]:
x = 30
y = 100
z = 100

num_epochs = 20
batch_size = 45

In [23]:
Input_shape = (30, 100, 100, 3)

model_2 = Sequential()
#first layer
model_2.add(Conv3D(16, (3,3,3), padding='same', input_shape = Input_shape))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D((2, 2,2), padding = "same"))

#2nd layer
model_2.add(Conv3D(32, (3, 3,3), padding = "same"))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D((2, 2,2), padding = "same"))

#3rd layer
model_2.add(Conv3D(64, (3, 3,3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D((2, 2,2), padding = "same"))

#4th layer
model_2.add(Conv3D(128, (3, 3,3), padding = "same"))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D((2, 2,2), padding = "same"))

model_2.add(Flatten())
model_2.add(Dense(256))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.25))

model_2.add(Dense(128))
model_2.add(Activation("relu"))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.25))

model_2.add(Dense(5))
model_2.add(Activation('softmax'))

In [24]:
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.001) #write your optimizer
model_2.compile(optimizer = optimiser, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print (model_2.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 100, 100, 16)  1312      
                                                                 
 activation_14 (Activation)  (None, 30, 100, 100, 16)  0         
                                                                 
 batch_normalization_12 (Bat  (None, 30, 100, 100, 16)  64       
 chNormalization)                                                
                                                                 
 max_pooling3d_8 (MaxPooling  (None, 15, 50, 50, 16)   0         
 3D)                                                             
                                                                 
 conv3d_9 (Conv3D)           (None, 15, 50, 50, 32)    13856     
                                                                 
 activation_15 (Activation)  (None, 15, 50, 50, 32)   

In [25]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [26]:
model_name = 'model_init_2' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = False, save_weights_only = False, mode = 'auto', save_freq = 'epoch')

LR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2, cooldown = 1, verbose = 1)
callbacks_list = [checkpoint, LR]

In [27]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [28]:
model_2.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs = num_epochs, verbose = 1, 
                    callbacks = callbacks_list, validation_data = val_generator, 
                    validation_steps = validation_steps, class_weight = None, workers = 1, initial_epoch = 0)

Source path =  Project_data/train ; batch size = 45
Epoch 1/20
15/15 [==============================] - ETA: 0s - loss: 1.4702 - categorical_accuracy: 0.4419 Source path =  Project_data/val ; batch size = 45

Epoch 1: saving model to model_init_2_2022-07-1206_46_11.972365/model-00001-1.47017-0.44193-48.60880-0.23000.h5
15/15 [==============================] - 635s 42s/step - loss: 1.4702 - categorical_accuracy: 0.4419 - val_loss: 48.6088 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/20
15/15 [==============================] - ETA: 0s - loss: 0.8781 - categorical_accuracy: 0.6667 
Epoch 2: saving model to model_init_2_2022-07-1206_46_11.972365/model-00002-0.87814-0.66667-26.96347-0.16667.h5
15/15 [==============================] - 462s 31s/step - loss: 0.8781 - categorical_accuracy: 0.6667 - val_loss: 26.9635 - val_categorical_accuracy: 0.1667 - lr: 0.0010
Epoch 3/20
15/15 [==============================] - ETA: 0s - loss: 0.7067 - categorical_accuracy: 0.7253 
Epoch 3: saving

# Model 3 

Inputs:
x = 30
y = 100
z = 100

Changing batch size and number of epochs:

batch_size = 45
epochs = 25

In [29]:
x = 30
y = 100
z = 100

num_epochs = 25
batch_size = 45

In [7]:
Input_shape = (30, 100, 100, 3)

model_3 = Sequential()
#first layer
model_3.add(Conv3D(16, (3,3,3), padding = 'same', input_shape = Input_shape))
model_3.add(Activation('relu'))
model_3.add(BatchNormalization())
model_3.add(MaxPooling3D((2, 2,2), padding = "same"))

#2nd layer
model_3.add(Conv3D(32, (3, 3,3), padding = "same"))
model_3.add(Activation('relu'))
model_3.add(BatchNormalization())
model_3.add(MaxPooling3D((2, 2,2), padding = "same"))
#model.add(Dropout(0.5))

#3rd layer
model_3.add(Conv3D(64, (3, 3,3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(BatchNormalization())
model_3.add(MaxPooling3D((2, 2,2), padding = "same"))

#4th layer
model_3.add(Conv3D(128, (3, 3,3), padding = "same"))
model_3.add(Activation('relu'))
model_3.add(BatchNormalization())
model_3.add(MaxPooling3D((2, 2,2), padding = "same"))

model_3.add(Flatten())
model_3.add(Dense(256))
model_3.add(Activation('relu'))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.25))

model_3.add(Dense(128))
model_3.add(Activation("relu"))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.25))

model_3.add(Dense(5))
model_3.add(Activation('softmax'))

In [31]:
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.001) #write your optimizer
model_3.compile(optimizer = optimiser, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print(model_3.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 30, 100, 100, 16)  1312      
                                                                 
 activation_21 (Activation)  (None, 30, 100, 100, 16)  0         
                                                                 
 batch_normalization_18 (Bat  (None, 30, 100, 100, 16)  64       
 chNormalization)                                                
                                                                 
 max_pooling3d_12 (MaxPoolin  (None, 15, 50, 50, 16)   0         
 g3D)                                                            
                                                                 
 conv3d_13 (Conv3D)          (None, 15, 50, 50, 32)    13856     
                                                                 
 activation_22 (Activation)  (None, 15, 50, 50, 32)   

In [32]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [33]:
model_name = 'model_init_3' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = False, save_weights_only = False, mode = 'auto', save_freq = 'epoch')

LR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2, cooldown = 1, verbose = 1)
callbacks_list = [checkpoint, LR]

In [34]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [35]:
model_3.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs = num_epochs, verbose = 1, 
                    callbacks = callbacks_list, validation_data = val_generator, 
                    validation_steps = validation_steps, class_weight = None, workers = 1, initial_epoch = 0)

Source path =  Project_data/train ; batch size = 45
Epoch 1/25
15/15 [==============================] - ETA: 0s - loss: 1.4570 - categorical_accuracy: 0.4419 Source path =  Project_data/val ; batch size = 45

Epoch 1: saving model to model_init_3_2022-07-1206_46_11.972365/model-00001-1.45702-0.44193-58.34675-0.21000.h5
15/15 [==============================] - 637s 42s/step - loss: 1.4570 - categorical_accuracy: 0.4419 - val_loss: 58.3468 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 2/25
15/15 [==============================] - ETA: 0s - loss: 0.9686 - categorical_accuracy: 0.6141 
Epoch 2: saving model to model_init_3_2022-07-1206_46_11.972365/model-00002-0.96857-0.61414-66.41541-0.16667.h5
15/15 [==============================] - 467s 31s/step - loss: 0.9686 - categorical_accuracy: 0.6141 - val_loss: 66.4154 - val_categorical_accuracy: 0.1667 - lr: 0.0010
Epoch 3/25
15/15 [==============================] - ETA: 0s - loss: 0.7718 - categorical_accuracy: 0.7172 
Epoch 3: saving

# Model 4


Inputs: 

Changing the image size from (100, 100) to (120, 120)

x = 30
y = 120
z = 120

Reducing the number of epochs:
batch_size = 40
num_epochs = 20

In [6]:
x = 30
y = 120
z = 120

batch_size = 40
num_epochs = 20

In [7]:
Input_shape = (30, 120, 120, 3)

model_4 = Sequential()
#first layer
model_4.add(Conv3D(16, (3,3,3), padding='same', input_shape = Input_shape))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D((2, 2,2), padding = "same"))

#2nd layer
model_4.add(Conv3D(32, (3, 3,3), padding = "same"))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D((2, 2,2), padding = "same"))

#3rd layer
model_4.add(Conv3D(64, (3, 3,3), padding='same'))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D((2, 2,2), padding = "same"))

#4th layer
model_4.add(Conv3D(128, (3, 3,3), padding = "same"))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D((2, 2,2), padding = "same"))

model_4.add(Flatten())
model_4.add(Dense(256))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())
model_4.add(Dropout(0.25))

model_4.add(Dense(128))
model_4.add(Activation("relu"))
model_4.add(BatchNormalization())
model_4.add(Dropout(0.25))

model_4.add(Dense(5))
model_4.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.001)
model_4.compile(optimizer = optimiser, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print(model_4.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 16)  1312      
                                                                 
 activation (Activation)     (None, 30, 120, 120, 16)  0         
                                                                 
 batch_normalization (BatchN  (None, 30, 120, 120, 16)  64       
 ormalization)                                                   
                                                                 
 max_pooling3d (MaxPooling3D  (None, 15, 60, 60, 16)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 60, 60, 32)    13856     
                                                                 
 activation_1 (Activation)   (None, 15, 60, 60, 32)    0

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init_4' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = False, save_weights_only = False, mode = 'auto', save_freq = 'epoch')

LR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2, cooldown = 1, verbose = 1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [13]:
model_4.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs = num_epochs, verbose = 1, 
                    callbacks = callbacks_list, validation_data = val_generator, 
                    validation_steps = validation_steps, class_weight = None, workers = 1, initial_epoch = 0)

Source path =  Project_data/train ; batch size = 40
Epoch 1/20
17/17 [==============================] - ETA: 0s - loss: 1.5644 - categorical_accuracy: 0.4570 Source path =  Project_data/val ; batch size = 40

Epoch 1: saving model to model_init_4_2022-07-1209_53_17.910281/model-00001-1.56435-0.45701-43.73214-0.23000.h5
17/17 [==============================] - 907s 53s/step - loss: 1.5644 - categorical_accuracy: 0.4570 - val_loss: 43.7321 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/20
17/17 [==============================] - ETA: 0s - loss: 1.0258 - categorical_accuracy: 0.6215 
Epoch 2: saving model to model_init_4_2022-07-1209_53_17.910281/model-00002-1.02577-0.62148-15.26606-0.28333.h5
17/17 [==============================] - 531s 31s/step - loss: 1.0258 - categorical_accuracy: 0.6215 - val_loss: 15.2661 - val_categorical_accuracy: 0.2833 - lr: 0.0010
Epoch 3/20
17/17 [==============================] - ETA: 0s - loss: 0.9269 - categorical_accuracy: 0.6394 
Epoch 3: saving

# Model 5


Using the same inputs but changing the dropout values to counter overfitting

In [6]:
x = 30
y = 120
z = 120

batch_size = 40
num_epochs = 20

In [7]:
Input_shape = (30, 120, 120, 3)

model_5 = Sequential()
#first layer
model_5.add(Conv3D(16, (3,3,3), padding='same', input_shape = Input_shape))
model_5.add(Activation('relu'))
model_5.add(BatchNormalization())
model_5.add(MaxPooling3D((2, 2,2), padding = "same"))

#2nd layer
model_5.add(Conv3D(32, (3, 3,3), padding = "same"))
model_5.add(Activation('relu'))
model_5.add(BatchNormalization())
model_5.add(MaxPooling3D((2, 2,2), padding = "same"))

#3rd layer
model_5.add(Conv3D(64, (3, 3,3), padding='same'))
model_5.add(Activation('relu'))
model_5.add(BatchNormalization())
model_5.add(MaxPooling3D((2, 2,2), padding = "same"))

#4th layer
model_5.add(Conv3D(128, (3, 3,3), padding = "same"))
model_5.add(Activation('relu'))
model_5.add(BatchNormalization())
model_5.add(MaxPooling3D((2, 2,2), padding = "same"))

model_5.add(Flatten())
model_5.add(Dense(256))
model_5.add(Activation('relu'))
model_5.add(BatchNormalization())
model_5.add(Dropout(0.5))

model_5.add(Dense(128))
model_5.add(Activation("relu"))
model_5.add(BatchNormalization())
model_5.add(Dropout(0.5))

model_5.add(Dense(5))
model_5.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.001) #write your optimizer
model_5.compile(optimizer = optimiser, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print(model_5.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 16)  1312      
                                                                 
 activation (Activation)     (None, 30, 120, 120, 16)  0         
                                                                 
 batch_normalization (BatchN  (None, 30, 120, 120, 16)  64       
 ormalization)                                                   
                                                                 
 max_pooling3d (MaxPooling3D  (None, 15, 60, 60, 16)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 60, 60, 32)    13856     
                                                                 
 activation_1 (Activation)   (None, 15, 60, 60, 32)    0

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init_5' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose = 1, save_best_only = False, save_weights_only = False, mode = 'auto', save_freq = 'epoch')

LR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2, cooldown = 1, verbose = 1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [12]:
model_5.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs = num_epochs, verbose = 1, 
                    callbacks = callbacks_list, validation_data = val_generator, 
                    validation_steps = validation_steps, class_weight = None, workers = 1, initial_epoch = 0)

Source path =  Project_data/train ; batch size = 40
Epoch 1/20
17/17 [==============================] - ETA: 0s - loss: 1.8920 - categorical_accuracy: 0.3439 Source path =  Project_data/val ; batch size = 40

Epoch 1: saving model to model_init_5_2022-07-1300_31_31.819012/model-00001-1.89204-0.34389-88.24358-0.23000.h5
17/17 [==============================] - 779s 46s/step - loss: 1.8920 - categorical_accuracy: 0.3439 - val_loss: 88.2436 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/20
17/17 [==============================] - ETA: 0s - loss: 1.3886 - categorical_accuracy: 0.5294 
Epoch 2: saving model to model_init_5_2022-07-1300_31_31.819012/model-00002-1.38860-0.52941-30.20901-0.28333.h5
17/17 [==============================] - 461s 27s/step - loss: 1.3886 - categorical_accuracy: 0.5294 - val_loss: 30.2090 - val_categorical_accuracy: 0.2833 - lr: 0.0010
Epoch 3/20
17/17 [==============================] - ETA: 0s - loss: 1.3347 - categorical_accuracy: 0.5038 
Epoch 3: saving

# Model using GRU (CNN + RNN)

In [13]:
#using the best model's parameters

x = 30
y = 120
z = 120

batch_size = 40
num_epochs = 20

In [14]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU,Dropout, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model_gru = Sequential()
Input_shape = (30, 120, 120, 3)

#first layer
model_gru.add(TimeDistributed(Conv2D(8, (3, 3),padding='same', activation='relu'),input_shape=Input_shape))
model_gru.add(TimeDistributed(BatchNormalization()))
model_gru.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_gru.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu')))
model_gru.add(TimeDistributed(BatchNormalization()))
model_gru.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_gru.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_gru.add(TimeDistributed(BatchNormalization()))
model_gru.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_gru.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_gru.add(TimeDistributed(BatchNormalization()))
model_gru.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_gru.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_gru.add(TimeDistributed(BatchNormalization()))
model_gru.add(TimeDistributed(MaxPooling2D((2, 2))))

# Flatten layer 
model_gru.add(TimeDistributed(Flatten()))

model_gru.add(GRU(64))
model_gru.add(Dropout(0.5))

# Dense layer 
model_gru.add(Dense(64, activation='relu'))
model_gru.add(Dropout(0.5))

# Softmax layer
model_gru.add(Dense(5, activation='softmax'))


In [15]:
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.001) #write your optimizer
model_gru.compile(optimizer = optimiser, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print(model_gru.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 30, 120, 120, 8)  224       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 30, 120, 120, 8)  32        
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 30, 60, 60, 8)    0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 30, 60, 60, 16)   1168      
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 30, 60, 60, 16)  

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [17]:
model_name = 'model_init_gru_1' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = False, save_weights_only = False, mode = 'auto', save_freq = 'epoch')

LR =ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2, cooldown = 1, verbose = 1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [19]:
model_gru.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs = num_epochs, verbose = 1, 
                    callbacks = callbacks_list, validation_data = val_generator, 
                    validation_steps = validation_steps, class_weight = None, workers = 1, initial_epoch = 0)

Source path =  Project_data/train ; batch size = 40
Epoch 1/20
17/17 [==============================] - ETA: 0s - loss: 1.7541 - categorical_accuracy: 0.2971Source path =  Project_data/val ; batch size = 40

Epoch 1: saving model to model_init_gru_1_2022-07-1300_31_31.819012/model-00001-1.75410-0.29713-2.38345-0.20000.h5
17/17 [==============================] - 101s 6s/step - loss: 1.7541 - categorical_accuracy: 0.2971 - val_loss: 2.3834 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 2/20
17/17 [==============================] - ETA: 0s - loss: 1.4940 - categorical_accuracy: 0.3734
Epoch 2: saving model to model_init_gru_1_2022-07-1300_31_31.819012/model-00002-1.49395-0.37340-2.33306-0.21667.h5
17/17 [==============================] - 59s 4s/step - loss: 1.4940 - categorical_accuracy: 0.3734 - val_loss: 2.3331 - val_categorical_accuracy: 0.2167 - lr: 0.0010
Epoch 3/20
17/17 [==============================] - ETA: 0s - loss: 1.4073 - categorical_accuracy: 0.4271
Epoch 3: saving m